# Multi-LLM Workflows

## Recap
You just learned function calling: how to connect LLMs to external systems. While function calling connects LLMs to external systems, these workflows show how to coordinate multiple LLM calls to solve complex problems.

## What This Covers
Single LLM calls often aren't enough. This notebook explores workflow patterns [inspired by Anthropic](https://www.anthropic.com/research/building-effective-agents):

1. **Prompt Chaining**: Sequential steps where each builds on the previous
2. **Parallelization**: Independent tasks running concurrently  
3. **Routing**: Dynamic selection of specialized paths
4. **Orchestrator-Workers**: Coordinating multiple specialized LLMs for complex tasks
5. **Evaluator-Optimizer**: Iterative refinement with feedback loops

**What you'll learn:** How to decompose complex problems into LLM workflows

**By the end:** You'll understand when and how to use each pattern for real-world applications.

## Setup

First, set your API key and import the necessary libraries:

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Callable
from util import llm_call, extract_xml


## 1. Prompt Chaining

Prompt chaining breaks complex tasks into sequential steps where each LLM call builds on the previous output.

![Prompt Chaining Workflow](img/prompt_chaining.png)
*Image from Anthropic*

### When to Use
- When a task naturally breaks down into sequential steps
- When each step's output feeds into the next step
- When you need clear intermediate results
- When order of operations matters

### Example: LinkedIn Profile Parser
This example demonstrates prompt chaining by:
1. First extracting structured data from a profile
2. Then using that structured data to generate a personalized email
3. Each step builds on the output of the previous step

In [2]:
# Example 1: Chain workflow for structured data extraction and formatting

def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    result = input
    for i, prompt in enumerate(prompts, 1):
        print(f"\nStep {i}:")
        result = llm_call(f"{prompt}\nInput: {result}")
        print(result)
    return result

def extract_structured_data(profile_text: str) -> str:
    """Extract all structured data from a LinkedIn profile in a single LLM call."""
    prompt = f"""
    Extract the following structured data from the LinkedIn profile text:
    - Full Name
    - Current Job Title and Company
    - Skills (as a comma-separated list)
    - Previous Job Titles (as a numbered list)

    Provide the output in this JSON format:
    {{
        "name": "Full Name",
        "current_position": "Position at Company",
        "skills": ["Skill1", "Skill2", ...],
        "previous_positions": ["Previous Position 1", "Previous Position 2", ...]
    }}

    LinkedIn Profile: {profile_text}
    """
    return llm_call(prompt)

def generate_outreach_email(data: str) -> str:
    """Generate a professional outreach email using the structured data."""
    prompt = f"""
    Using the following structured data, write a professional outreach email:
    {data}
    
    The email should:
    - Address the recipient by name.
    - Reference their current position and company.
    - Highlight relevant skills.
    - Politely request a meeting to discuss potential collaboration opportunities.
    """
    return llm_call(prompt)

In [3]:
# Example LinkedIn profile input
linkedin_profile_elliot = """
Elliot Alderson is a Cybersecurity Engineer at Allsafe Security. He specializes in penetration testing, network security, and ethical hacking.
Elliot has a deep understanding of UNIX systems, Python, and C, and is skilled in identifying vulnerabilities in corporate networks.
In his free time, Elliot is passionate about open-source projects and contributing to cybersecurity forums.
Previously, he worked as a freelance cybersecurity consultant, assisting clients in securing their online assets.
"""



Now let's execute the chain:

In [4]:
structured_data = extract_structured_data(linkedin_profile_elliot)
print("\nExtracted Structured Data:")
print(structured_data)




Extracted Structured Data:
```json
{
    "name": "Elliot Alderson",
    "current_position": "Cybersecurity Engineer at Allsafe Security",
    "skills": ["penetration testing", "network security", "ethical hacking", "UNIX systems", "Python", "C", "identifying vulnerabilities", "open-source projects", "cybersecurity"],
    "previous_positions": ["Freelance Cybersecurity Consultant"]
}
```


Now generate the outreach email:

In [5]:
email = generate_outreach_email(structured_data)
print("\nGenerated Outreach Email:")
print(email)


Generated Outreach Email:
Subject: Exploring Cybersecurity Collaboration Opportunities

Dear Elliot,

I hope this email finds you well. I'm reaching out to you in your role as Cybersecurity Engineer at Allsafe Security, as I believe there may be valuable opportunities for us to collaborate.

I've been impressed by your extensive expertise in cybersecurity, particularly your skills in penetration testing, network security, and ethical hacking. Your proficiency with UNIX systems and programming languages like Python and C, combined with your talent for identifying vulnerabilities, represents exactly the kind of technical depth that aligns with potential projects I have in mind.

Given your background as a Freelance Cybersecurity Consultant and your continued work in the field, I believe your perspective would be invaluable. I'm particularly interested in discussing how your experience with open-source projects and comprehensive approach to cybersecurity could complement some initiatives

## 2. Parallelization

Parallelization runs independent LLM calls concurrently to speed up processing.

![Parallelization Workflow](img/parallelization_workflow.png)

*Image from Anthropic*

### When to Use
- When different aspects of a task can be processed independently
- When you need to analyze multiple components simultaneously
- When speed/performance is a priority
- When you have multiple similar items to process (like batch processing)


### Example: LinkedIn Profile Field Extraction
This example demonstrates parallelization by:
1. Simultaneously extracting different fields from a profile:
   - Name extraction
   - Position and company extraction
   - Skills extraction
2. Using ThreadPoolExecutor to manage concurrent LLM calls
3. Combining the parallel extractions into a unified profile view

In [6]:
def parallel(prompt: str, inputs: List[str], n_workers: int = 3) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}\nInput: {x}") for x in inputs]
        return [f.result() for f in futures]

In [7]:
# Define independent extraction tasks that will run in parallel
field_extraction_prompts = [
    """Extract the full name from the following LinkedIn profile text. Return only the name.
    LinkedIn Profile: {input}""",
    
    """Extract the current job title and company from the following LinkedIn profile text.
    Format as:
    Position: [Job Title]
    Company: [Company Name]
    LinkedIn Profile: {input}""",
    
    """Extract the skills mentioned in the following LinkedIn profile text. Return them as a comma-separated list.
    LinkedIn Profile: {input}""",
    
    """Extract the previous job titles from the following LinkedIn profile text. Return them as a numbered list, one per line.
    LinkedIn Profile: {input}"""
]

Execute the parallel extractions:

In [8]:
# Process all field extractions in parallel
extracted_fields = parallel(
    """Perform the following field extraction task:
    {input}""",
    [prompt.replace("{input}", linkedin_profile_elliot) for prompt in field_extraction_prompts]
)

# Assign extracted results to field names for clarity
field_names = ["Full Name", "Current Position and Company", "Skills", "Previous Positions"]
structured_data = {field: result for field, result in zip(field_names, extracted_fields)}

# Combine extracted fields into a JSON object
structured_data_json = {
    "name": structured_data["Full Name"],
    "current_position": structured_data["Current Position and Company"],
    "skills": structured_data["Skills"].split(", "),
    "previous_positions": structured_data["Previous Positions"].split("\n")
}

print("\nExtracted Structured Data (JSON):")
print(structured_data_json)


Extracted Structured Data (JSON):
{'name': 'Elliot Alderson', 'current_position': 'Position: Cybersecurity Engineer\nCompany: Allsafe Security', 'skills': ['Based on the LinkedIn profile text', 'here are the extracted skills:\n\npenetration testing', 'network security', 'ethical hacking', 'UNIX systems', 'Python', 'C', 'vulnerability identification', 'cybersecurity consulting', 'open-source projects'], 'previous_positions': ['Based on the LinkedIn profile text, here are the previous job titles:', '', '1. Freelance Cybersecurity Consultant', '', 'Note: The current position mentioned is "Cybersecurity Engineer at Allsafe Security," which is not included in this list as it represents the present role, not a previous one.']}


Now generate an email using the parallel-extracted data:

In [9]:
# Create the email
email = generate_outreach_email(structured_data_json)

print("\nGenerated Outreach Email:")
print(email)


Generated Outreach Email:
Subject: Exploring Collaboration Opportunities in Cybersecurity

Dear Elliot,

I hope this email finds you well. I came across your profile and was impressed by your work as a Cybersecurity Engineer at Allsafe Security.

Your expertise in penetration testing, network security, and ethical hacking particularly caught my attention, along with your strong technical foundation in UNIX systems, Python, and C. Your experience in vulnerability identification and cybersecurity consulting, combined with your contributions to open-source projects, demonstrates a comprehensive skill set that aligns well with the evolving challenges in our industry.

I believe there may be valuable opportunities for us to collaborate, and I would welcome the chance to discuss how our respective expertise might complement each other. Would you be available for a brief meeting in the coming weeks? I'm happy to work around your schedule and can meet virtually or in person, whichever you pre

## 3. Routing

Routing classifies inputs and directs them to specialized handling paths based on their type.

![Routing Workflow](img/routing_workflow.png)
*Image from Anthropic*

### When to Use
- When input types require different specialized handling
- When you need to direct tasks to specific LLM prompts
- When input classification determines the processing path
- When you have clearly defined categories of requests

### Example: LinkedIn Profile Classification
This example demonstrates routing by:
1. Analyzing profiles to determine if they are:
    - Individual profiles (for hiring outreach)
    - Company profiles (for business development)
2. Using different email templates based on classification
3. Ensuring appropriate tone and content for each type

Let's implement the routing pattern:

In [10]:
# Define email routes for different profile types
email_routes = {
    "hiring": """You are a talent acquisition specialist. Write a professional email inviting the individual to discuss career opportunities. 
    Highlight their skills and current position. Maintain a warm and encouraging tone.

    Input: """,
    
    "collaboration": """You are a business development specialist. Write a professional email proposing a collaboration with the company. 
    Highlight mutual benefits and potential opportunities. Maintain a formal yet friendly tone.

    Input: """
}

Define the routing function that classifies and delegates:

In [11]:
def route_linkedin_profile(input: str, routes: Dict[str, str]) -> str:
    """Route LinkedIn profile to the appropriate email generation task."""
    print(f"\nAvailable routes: {list(routes.keys())}")
    selector_prompt = f"""
    Analyze the following LinkedIn profile and classify it as:
    - "hiring" if it represents an individual suitable for talent outreach.
    - "collaboration" if it represents a company profile suitable for business development outreach.

    Provide your reasoning in plain text, and then your decision in this format:

    <reasoning>
    Brief explanation of why this profile was classified into one of the routes. 
    Consider key signals like job titles, skills, organizational descriptions, and tone.
    </reasoning>

    <selection>
    The chosen route name
    </selection>

    Profile: {input}
    """
    # Call the LLM for classification
    route_response = llm_call(selector_prompt)

    # Extract reasoning and route selection
    reasoning = extract_xml(route_response, "reasoning")
    route_key = extract_xml(route_response, "selection").strip().lower()

    print("\nRouting Analysis:")
    print(reasoning)

    # Handle invalid classifications
    if route_key not in routes:
        print(f"Invalid classification '{route_key}', defaulting to 'hiring'")
        route_key = "hiring"

    # Route to the appropriate email template
    selected_prompt = routes[route_key]
    return llm_call(f"{selected_prompt}\nProfile: {input}")

Define example profile to test the router:

In [12]:
# Company profile
linkedin_profile_company = """
E Corp is a global leader in technology and financial services. With a portfolio spanning software development, cloud infrastructure,
and consumer banking, E Corp serves millions of customers worldwide. Our mission is to deliver innovative solutions that drive
efficiency and growth for businesses and individuals alike. Learn more at www.ecorp.com.
"""

Route and generate email for the individual (Elliot from earlier):

In [13]:
print("Processing Individual Profile:")
email_individual = route_linkedin_profile(linkedin_profile_elliot, email_routes)
print("\nGenerated Email:")
print(email_individual)

Processing Individual Profile:

Available routes: ['hiring', 'collaboration']

Routing Analysis:

This profile represents an individual professional, not a company. Key indicators include:

1. **Individual identity**: "Elliot Alderson" is clearly a person's name, not a company name
2. **Job title and employer**: Listed as "Cybersecurity Engineer at Allsafe Security" - this shows an employee-employer relationship
3. **Personal skills and expertise**: The profile lists individual technical skills (penetration testing, Python, C, UNIX systems)
4. **Personal interests**: Mentions "his free time" and personal passion for open-source projects
5. **Career history**: References previous work as a "freelance cybersecurity consultant"
6. **Tone**: Written in third person about an individual's professional background and capabilities

This is clearly a personal professional profile of someone working in cybersecurity. As an individual contributor with specialized technical skills, this person wou

Route and generate email for the company profile:

In [14]:
print("Processing Company Profile:")
email_company = route_linkedin_profile(linkedin_profile_company, email_routes)
print("\nGenerated Email:")
print(email_company)

Processing Company Profile:

Available routes: ['hiring', 'collaboration']

Routing Analysis:

This profile represents a company rather than an individual. Key signals include:

1. **Organizational language**: The profile uses "E Corp is" and refers to itself as a corporate entity with "our mission," indicating this is a company page, not a personal profile.

2. **Company-level description**: It describes a broad portfolio of services (software development, cloud infrastructure, consumer banking) and mentions serving "millions of customers worldwide" - typical of corporate positioning statements.

3. **Lack of individual identifiers**: There are no personal job titles, individual skills, career history, or personal achievements mentioned that would indicate this is someone's professional profile.

4. **Business development opportunity**: As a "global leader in technology and financial services," this represents a potential partner or client for B2B collaboration, making it suitable for

## 4. Orchestrator-Workers


![alt text](img/orchestrator-worker.png "Title")
*Image from Anthropic*

An orchestrator coordinates multiple specialized LLMs (workers) for complex tasks. The orchestrator classifies inputs, delegates to appropriate workers, and can synthesize results.

### When to Use
- Complex tasks requiring multiple specialized steps
- Different workers handle different aspects (classification, generation, validation)
- Dynamic routing based on intermediate results

### Example: Industry-Specific Outreach
This orchestrator:
1. Classifies a LinkedIn profile by industry (Tech vs Non-Tech)
2. Routes to specialized workers that generate industry-appropriate emails



Let's implement the orchestrator-worker pattern. First, define the email routes:

In [15]:
# Define the email generation routes for different industries
email_routes = {
    "tech": """You are a talent acquisition specialist in the tech industry. Write a professional email to the individual described below, inviting them to discuss career opportunities in the tech field.
    Highlight their skills and current position. Maintain a warm and encouraging tone.

    Input: {profile_text}""",

    "non_tech": """You are a talent acquisition specialist. Write a professional email to the individual described below, inviting them to discuss career opportunities.
    Highlight their skills and current position in a non-tech field. Maintain a warm and encouraging tone.

    Input: {profile_text}"""
}

Define the classification function:

In [16]:
def llm_classify(input: str) -> str:
    """Use LLM to classify the industry of the profile (Tech or Not Tech)."""
    classify_prompt = f"""
    Analyze the LinkedIn profile below and classify the industry as either Tech or Not Tech.
    
    LinkedIn Profile: {input}
    """
    classification = llm_call(classify_prompt)
    return classification.strip().lower()

Define the orchestrator and worker functions:

In [17]:
def orchestrator(input: str, routes: Dict[str, str]) -> str:
    """Classify the LinkedIn profile and assign tasks to workers based on the classification."""
    # Classify the profile industry
    industry = llm_classify(input)
    print(f"\nClassified industry as: {industry.capitalize()}")

    # Route to the appropriate worker
    if industry == "tech":
        task_responses = [tech_worker(input, routes)]
    else:
        task_responses = [non_tech_worker(input, routes)]
    
    return task_responses

def tech_worker(input: str, routes: Dict[str, str]) -> str:
    """Generate the email for Tech industry profiles."""
    selected_prompt = routes["tech"]
    return llm_call(selected_prompt.format(profile_text=input))

def non_tech_worker(input: str, routes: Dict[str, str]) -> str:
    """Generate the email for Non-Tech industry profiles."""
    selected_prompt = routes["non_tech"]
    return llm_call(selected_prompt.format(profile_text=input))

Define an example profile:

Process a profile through the orchestrator:

In [18]:
print("Processing Individual Profile (Elliot Alderson):")
email_responses_individual = orchestrator(linkedin_profile_elliot, email_routes)
print("\nGenerated Email:")
for response in email_responses_individual:
    print(response)

Processing Individual Profile (Elliot Alderson):

Classified industry as: **classification: tech**

**reasoning:**

this linkedin profile clearly falls within the tech industry based on multiple indicators:

1. **job role**: cybersecurity engineer is a core technology position
2. **technical skills**: mentions specific technical competencies including:
   - penetration testing
   - network security
   - ethical hacking
   - unix systems
   - programming languages (python, c)

3. **industry focus**: cybersecurity is a specialized sector within the technology industry

4. **work activities**: identifying vulnerabilities in corporate networks and securing online assets are technology-focused responsibilities

5. **professional interests**: contributing to open-source projects and cybersecurity forums are activities firmly rooted in the tech community

this profile represents a professional working directly in information technology and cybersecurity, which is unequivocally part of the tec

Let's run the orchestrator on several different profiles to see how it classifies and routes them:

In [19]:
# Example LinkedIn profiles across Industries (for orchestrator-workers workflow)

# Tony Stark (Engineering - Entertainment/Tech Industry)
linkedin_profile_tony_stark = """
Tony Stark is the CEO of Stark Industries and a renowned inventor and engineer. He specializes in advanced robotics, artificial intelligence, and defense technologies.
Tony is best known for creating the Iron Man suit and leading innovations in the field of clean energy. He has a passion for pushing the boundaries of science and technology to protect humanity.
Previously, Tony Stark served as an inventor and entrepreneur, having founded Stark Industries and revolutionized the defense industry.
"""

# Sheryl Sandberg (Business - Tech Industry)
linkedin_profile_sheryl_sandberg = """
Sheryl Sandberg is the Chief Operating Officer at Facebook (Meta), specializing in business operations, scaling organizations, and team management.
She has a strong background in strategic planning, marketing, and organizational leadership. Previously, Sheryl served as Vice President of Global Online Sales and Operations at Google.
She is also the author of *Lean In*, a book focused on empowering women in leadership positions.
"""

# Elon Musk (Entrepreneur - Tech/Space Industry)
linkedin_profile_elon_musk = """
Elon Musk is the CEO of SpaceX and Tesla, Inc. He is an entrepreneur and innovator with a focus on space exploration, electric vehicles, and renewable energy.
Musk's work has revolutionized the automotive industry with Tesla’s electric vehicles and space exploration with SpaceX’s reusable rockets. He is also the founder of The Boring Company and Neuralink.
Musk is dedicated to advancing sustainable energy solutions and enabling human life on Mars.
"""

# Walter White (Chemistry - Entertainment/Film Industry)
linkedin_profile_walter_white = """
Walter White is a former high school chemistry teacher turned chemical engineer, best known for his work in the methamphetamine production industry.
Initially, Walter worked as a chemistry professor at a university before turning to a life of crime to secure his family's future.
Over time, he became an expert in chemical processes and synthesis, and his work has had profound impacts on the illegal drug trade. He is currently retired and focusing on his personal legacy.
"""

# Hermione Granger (Education - Literary/Film Industry)
linkedin_profile_hermione_granger = """
Hermione Granger is a research specialist at the Department of Magical Research and Development, focusing on magical education and the preservation of magical history.
She specializes in spellcraft, magical law, and potion-making. Hermione has worked closely with the Ministry of Magic to develop educational programs for young witches and wizards.
In her earlier years, she attended Hogwarts School of Witchcraft and Wizardry, where she excelled in every subject. She's passionate about equal rights for magical creatures and is an advocate for social justice.
"""

In [20]:
# Process the LinkedIn profiles and generate emails
profiles = [
    linkedin_profile_elliot,
    linkedin_profile_tony_stark, linkedin_profile_sheryl_sandberg,
    linkedin_profile_elon_musk, linkedin_profile_walter_white,
    linkedin_profile_hermione_granger
]

# Process each profile
for profile in profiles:
    print("\nProcessing LinkedIn Profile:")
    email_responses = orchestrator(profile, email_routes)
    print("\nGenerated Emails:")
    for response in email_responses:
        print(response)


Processing LinkedIn Profile:

Classified industry as: **classification: tech**

**reasoning:**

this linkedin profile clearly falls within the tech industry based on multiple indicators:

1. **job role**: cybersecurity engineer is a core technology position
2. **technical skills**: mentions specific technical competencies including:
   - penetration testing
   - network security
   - ethical hacking
   - unix systems
   - programming languages (python, c)

3. **industry focus**: cybersecurity is a specialized sector within the technology industry

4. **work activities**: identifying vulnerabilities in corporate networks and securing online assets are technology-focused responsibilities

5. **professional interests**: contributing to open-source projects and cybersecurity forums are hallmarks of tech industry engagement

this profile represents a quintessential technology professional working in the cybersecurity domain.

Generated Emails:
**Subject: Exciting Career Opportunity for an 

## 5. Evaluator-Optimizer

An evaluator-optimizer workflow uses iterative refinement. A generator creates output, an evaluator assesses it, and an optimizer improves it based on feedback.

![Evaluator-Optimizer Workflow](img/evaluator-optimizer.png)
*Image from Anthropic*

### When to Use
- Iterative improvement needed to meet quality criteria
- Clear evaluation standards available
- Task benefits from feedback loops

### Example: Refining Outreach Emails
This workflow:
1. Generates an initial email from a LinkedIn profile
2. Evaluates it for professionalism, tone, and clarity
3. Optimizes based on feedback until quality criteria are met

Let's implement the evaluator-optimizer pattern. First, define the email route:

In [21]:
# Define the email generation route
email_routes = {
    "hiring": """You are a talent acquisition specialist. Write a professional email to the individual described below, inviting them to discuss career opportunities. 
    Highlight their skills and current position. Maintain a warm and encouraging tone.

    Input: {profile_text}"""
}

Define the email generator function:

In [22]:
def llm_generate_email(input: str, routes: Dict[str, str]) -> str:
    """Generate an email based on the LinkedIn profile."""
    selected_prompt = routes["hiring"]
    return llm_call(selected_prompt.format(profile_text=input))

Define the evaluator function:

In [23]:
def llm_evaluate_email(email: str) -> str:
    """Evaluate the generated email for professionalism, tone, and clarity."""
    evaluation_prompt = f"""
    Please review the following email and provide feedback.
    The goal is to ensure it is professional, clear, and maintains a warm tone. If it needs improvements, provide suggestions.

    Email: {email}
    """
    return llm_call(evaluation_prompt)

Define the optimizer function:

In [24]:
def llm_optimize_email(email: str, feedback: str) -> str:
    """Refine the generated email based on evaluator feedback."""
    optimization_prompt = f"""
    Based on the following feedback, improve the email. Ensure it remains professional and clear while implementing the suggested changes.

    Feedback: {feedback}
    Email: {email}
    """
    return llm_call(optimization_prompt)

Define the orchestrator that coordinates the generator, evaluator, and optimizer:

In [25]:
def eval_optimizer_workflow(input: str, routes: Dict[str, str]) -> str:
    """Generate, evaluate, and optimize the email."""
    # Step 1: Generate the initial email
    email = llm_generate_email(input, routes)
    print("\nInitial Generated Email:")
    print(email)

    # Step 2: Evaluate the email
    feedback = llm_evaluate_email(email)
    print("\nEvaluator Feedback:")
    print(feedback)

    # Step 3: Optimize the email based on feedback
    optimized_email = llm_optimize_email(email, feedback)
    print("\nOptimized Email:")
    print(optimized_email)

    return optimized_email

Process Elliot's profile:

In [26]:
print("Processing LinkedIn Profile (Elliot Alderson):")
final_email = eval_optimizer_workflow(linkedin_profile_elliot, email_routes)

print("\nFinal Optimized Email:")
print(final_email)

Processing LinkedIn Profile (Elliot Alderson):

Initial Generated Email:
**Subject:** Exciting Career Opportunity for an Exceptional Cybersecurity Professional

Dear Elliot,

I hope this message finds you well.

My name is [Your Name], and I'm a Talent Acquisition Specialist at [Company Name]. I came across your profile and was genuinely impressed by your expertise and accomplishments in the cybersecurity field.

Your current role as a Cybersecurity Engineer at Allsafe Security, combined with your specialized skills in penetration testing, network security, and ethical hacking, demonstrates a level of technical excellence that's rare to find. Your deep understanding of UNIX systems and proficiency in Python and C, along with your proven ability to identify and address vulnerabilities in corporate networks, speaks volumes about your capabilities.

What particularly caught my attention is your passion for the cybersecurity community—your contributions to open-source projects and active p

## Summary

You've now seen five workflow patterns for coordinating multiple LLM calls:

1. **Prompt Chaining**: Sequential steps building on each other
2. **Parallelization**: Independent tasks running concurrently
3. **Routing**: Dynamic path selection based on input type
4. **Orchestrator-Workers**: Central coordinator delegating to specialized workers
5. **Evaluator-Optimizer**: Iterative refinement through feedback loops

**Key takeaway:** Complex AI applications rarely use a single LLM call. These patterns help you structure multi-LLM workflows for reliability, efficiency, and quality.